In [1]:
from pymongo import MongoClient
from pprint import pprint as pp
client = MongoClient('192.168.1.220', 27017)
db = client['test']

In [34]:
### 初始購物清單
db.bar.delete_one({'_id': 123})

db.bar.insert_one({
    '_id' : 123,
    'total' : 10,
    'items' : [
        { 'name' : "A", 'price' : 2 },
        { 'name' : "B", 'price' : 3 },
        { 'name' : "C", 'price' : 5 }
    ]
})

pp(db.bar.find_one({}, {'_id': 0}))

{'items': [{'name': 'A', 'price': 2},
           {'name': 'B', 'price': 3},
           {'name': 'C', 'price': 5}],
 'total': 10}


In [18]:
order = 123
product = "A"
price = 5

In [35]:
### 購物清單中 移除一個商品

# 1. 移除 特定名字那包
cur = db.bar.update_one({'_id': order, "items.name": { '$eq': product }}, 
                        { '$pull': { 'items': { 'name': product }}})
# update 用來刪除 Array 內的 {}

# 2. 更改 total price
total_price = db.bar.aggregate([
    {'$match': {'_id': order}},
    {'$project': {'_id': 0, 'total': {'$sum': '$items.price'}}}
]).next()['total']

db.bar.update_one(
    {'_id' : order},
    {'$set': { "total": total_price}}
)

{'items': [{'name': 'B', 'price': 3}, {'name': 'C', 'price': 5}], 'total': 8}


In [39]:
### 查詢變更結果
pp(db.bar.find_one({}, {'_id': 0}))

{'items': [{'name': 'B', 'price': 3},
           {'name': 'C', 'price': 5},
           {'name': 'A', 'price': 5}],
 'total': 13}


In [38]:
### 購物清單中 增加一個商品

# 1. 增加商品到購物清單
res = db.bar.update_one({'_id': order, "items.name": { '$ne': product }}, 
                        { '$addToSet': { 'items': { 'name': product }}})

# 2. 
if res.modified_count > 0:  # 新增一包 (api的 post作的)
    db.bar.update_one(
        {'_id': order, "items.name": product}, 
        { 
            '$set': { "items.$.price": price},
            '$inc': { 'total' : price }
        }
    )
    
else: # 修改已有的那包 (api的 put作的)
    """
        流程:
        1. 把新增的物件增加價格欄位
        2. 重新計算 total_price
        3. 更新 total欄位 = total_price
    """
    
    # 1.
    db.bar.update_one(
        {'_id': order, "items.name": product}, 
        {'$set': { "items.$.price": price}}
    )
    
    # 2.
    total_price = db.bar.aggregate([
        {'$match': {'_id': order}},
        {'$project': {'_id': 0, 'total': {'$sum': '$items.price'}}}
    ]).next()['total']
    
    # 3.
    db.bar.update_one(
        {'_id': order}, 
        {'$set': { "total": total_price}}
    )